In [ ]:
import numpy as np
import pandas as pd
from retry import retry
from filterpy.kalman import KalmanFilter
import numpy as np

In [ ]:
from tqdm import tqdm

In [ ]:
from nba_api.stats.endpoints import teamestimatedmetrics, boxscoreadvancedv3

In [ ]:
@retry()
def get_estimated_metrics(season="2023-24"):
    return teamestimatedmetrics.TeamEstimatedMetrics(season=season).get_data_frames()[0]

In [ ]:
def get_team_metrics(team_full_name, data):
  ratings = data[data["TEAM_NAME"] == team_full_name][["E_OFF_RATING", "E_DEF_RATING"]]

  out = {
      "E_OFF_RATING": ratings["E_OFF_RATING"].values[0],
      "E_DEF_RATING": ratings["E_DEF_RATING"].values[0]
  }

  return out

In [ ]:
def get_league_avg_ratings(data):
  return np.mean(data["E_OFF_RATING"])

In [ ]:
@retry()
def get_boxscore(game_id):
  return boxscoreadvancedv3.BoxScoreAdvancedV3(game_id=game_id).get_data_frames()[1]

In [ ]:
def get_matchup_ratings(team_A_name, team_A_abbrev, season="2023-24"):
  game_logs = pd.read_csv("game_log.csv")
  matchup_logs = game_logs[game_logs["TEAM_NAME"] == team_A_name]
  game_ids = matchup_logs["GAME_ID"].tolist()

  team_A_off_rtgs = []
  team_A_def_rtgs = []
  team_B_off_rtgs = []
  team_B_def_rtgs = []

  for i in tqdm(game_ids):
    fixed_id = "00" + str(i)
    game_data = get_boxscore(fixed_id)
    A_game_data = game_data[game_data["teamTricode"] == team_A_abbrev]
    B_game_data = game_data[game_data["teamTricode"] != team_A_abbrev]
    team_A_off_rtgs.append(float(A_game_data["offensiveRating"].iloc[0]))
    team_A_def_rtgs.append(float(A_game_data["defensiveRating"].iloc[0]))
    team_B_off_rtgs.append(float(B_game_data["offensiveRating"].iloc[0]))
    team_B_def_rtgs.append(float(B_game_data["defensiveRating"].iloc[0]))

  out = {
      "TEAM_A_OFF_RATING": team_A_off_rtgs,
      "TEAM_A_DEF_RATING": team_A_def_rtgs,
      "TEAM_B_OFF_RATING": team_B_off_rtgs,
      "TEAM_B_DEF_RATING": team_B_def_rtgs
  }

  return pd.DataFrame(out)



In [ ]:
def linear_prediction(team_A_off_rtg, team_A_def_rtg, team_B_off_rtg, team_B_def_rtg, league_avg):
  return ((team_A_off_rtg)*(team_B_def_rtg)) / league_avg, ((team_A_def_rtg)*(team_B_off_rtg)) / league_avg

In [ ]:
def calculate_updated_off_rating(ORA_minus, ORAG, DRB_minus, R, prior_variance):
  KORA = compute_KORA(prior_variance, ORA_minus, DRB_minus, R)
  #print(ORA_minus)
  #print(type(ORA_minus))
  ORA_plus = ORA_minus + KORA * ((ORAG - (ORA_minus * DRB_minus)) / R)
  var_ORA_plus = (1 - KORA) * prior_variance
  return ORA_plus, var_ORA_plus

In [ ]:
def compute_SE2(prior_variance, ORA_minus, DRB_minus, R):
  num = ((np.power(prior_variance, 2)) + ((np.power(ORA_minus, 2))*(prior_variance)) + ((np.power(DRB_minus, 2)) * prior_variance))
  return num / np.power(R, 2)

In [ ]:
def compute_KORA(prior_variance, ORA_minus, DRB_minus, R):
  SE2 = compute_SE2(prior_variance, ORA_minus, DRB_minus, R)
  SG2 = 100

  return SE2 / (SG2 + SE2)

In [ ]:
def compute_kORTG(team_full_name, team_abbrev, season="2023-24", prior_variance=20):
  emets = get_estimated_metrics()
  t_mets = get_team_metrics(team_full_name, emets)
  R = get_league_avg_ratings(emets)

  matchups = get_matchup_ratings(team_full_name, team_abbrev, season=season)
  ORA_minus = t_mets["E_OFF_RATING"]
  DRB_minus = t_mets["E_DEF_RATING"]

  for index, row in tqdm(matchups.iterrows()):
    ORA_minus, discard = linear_prediction(ORA_minus, DRB_minus, row["TEAM_B_OFF_RATING"], row["TEAM_B_DEF_RATING"], R)
    ORA_minus, prior_variance = calculate_updated_off_rating(ORA_minus, row["TEAM_A_OFF_RATING"], DRB_minus, R, prior_variance)

  return ORA_minus, prior_variance


In [ ]:
off_rtg, prior_var = compute_kORTG("Cleveland Cavaliers", "CLE")

In [ ]:
off_rtg

154.97711090620479

In [ ]:
prior_var

0.3181081385464605

In [ ]:
nba_teams = {
  "ATL": "Atlanta Hawks",
  "BOS": "Boston Celtics",
  "BKN": "Brooklyn Nets",
  "CHA": "Charlotte Hornets",
  "CHI": "Chicago Bulls",
  "CLE": "Cleveland Cavaliers",
  "DAL": "Dallas Mavericks",
  "DEN": "Denver Nuggets",
  "DET": "Detroit Pistons",
  "GSW": "Golden State Warriors",
  "HOU": "Houston Rockets",
  "IND": "Indiana Pacers",
  "LAC": "Los Angeles Clippers",
  "LAL": "Los Angeles Lakers",
  "MEM": "Memphis Grizzlies",
  "MIA": "Miami Heat",
  "MIL": "Milwaukee Bucks",
  "MIN": "Minnesota Timberwolves",
  "NOP": "New Orleans Pelicans",
  "NYK": "New York Knicks",
  "OKC": "Oklahoma City Thunder",
  "ORL": "Orlando Magic",
  "PHI": "Philadelphia 76ers",
  "PHX": "Phoenix Suns",
  "POR": "Portland Trail Blazers",
  "SAC": "Sacramento Kings",
  "SAS": "San Antonio Spurs",
  "TOR": "Toronto Raptors",
  "UTA": "Utah Jazz",
  "WAS": "Washington Wizards"
}